In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from vllm import LLM, SamplingParams
import argparse
import json
import os
import sys
# sys.path.append(os.path.abspath("/data/shuozhe/llm_reason/X-R1/src/x_r1"))
# from grpo import SYSTEM_PROMPT
# from rewards import accuracy_answer_reward
# import torch
import re
from transformers import AutoTokenizer
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse
from openai import OpenAI


/data/shuozhe/miniconda3/envs/xr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-19 05:08:00 __init__.py:190] Automatically detected platform cuda.


2025-06-19 05:08:00,781	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# model_name = "/teamspace/studios/this_studio/cot-faithfulness/output/Qwen2.5-3B_1e4_KL/checkpoint-187"
# model_name = "/teamspace/studios/this_studio/cot-faithfulness/output/Qwen2.5-3B/checkpoint-187"
# model_name = "/teamspace/studios/this_studio/saved_models/Qwen/Qwen2.5-3B-Instruct"

# model_name = "/data/shuozhe/saved_model/Open-RS3"
model_name = "/data/shuozhe/saved_model/DeepSeek-R1-Distill-Qwen-1.5B"

num_gpus = 4
llm = None
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create LLM object
llm = LLM(model=model_name,  # replace your own model
            dtype='bfloat16',
            tensor_parallel_size=num_gpus,  # number of gpu
            gpu_memory_utilization=0.5,  # prevent OOM
            trust_remote_code=True,
            # use_cache=False,
            )

INFO 06-19 05:08:10 config.py:542] This model supports multiple tasks: {'classify', 'generate', 'reward', 'embed', 'score'}. Defaulting to 'generate'.
INFO 06-19 05:08:10 config.py:1401] Defaulting to use mp for distributed inference
WARNING 06-19 05:08:10 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 06-19 05:08:10 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 06-19 05:08:10 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/data/shuozhe/saved_model/DeepSeek-R1-Distill-Qwen-1.5B', speculative_config=None, tokenizer='/data/shuozhe/saved_model/DeepSeek-R1-Distill-Qwen-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remot

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.22it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.21it/s]



(VllmWorkerProcess pid=3157147) INFO 06-19 05:08:14 model_runner.py:1115] Loading model weights took 0.8988 GB
(VllmWorkerProcess pid=3157144) INFO 06-19 05:08:14 model_runner.py:1115] Loading model weights took 0.8988 GB
INFO 06-19 05:08:14 model_runner.py:1115] Loading model weights took 0.8988 GB
(VllmWorkerProcess pid=3157152) INFO 06-19 05:08:14 model_runner.py:1115] Loading model weights took 0.8988 GB
(VllmWorkerProcess pid=3157144) (VllmWorkerProcess pid=3157152) (VllmWorkerProcess pid=3157147) INFO 06-19 05:08:24 worker.py:267] Memory profiling takes 9.72 seconds
INFO 06-19 05:08:24 worker.py:267] Memory profiling takes 9.57 seconds
INFO 06-19 05:08:24 worker.py:267] Memory profiling takes 9.75 seconds
(VllmWorkerProcess pid=3157144) (VllmWorkerProcess pid=3157152) (VllmWorkerProcess pid=3157147) INFO 06-19 05:08:24 worker.py:267] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.50) = 11.84GiB
INFO 06-19 05:08:24 worker.py:267] the curr

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

(VllmWorkerProcess pid=3157144) INFO 06-19 05:08:30 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.37it/s]

INFO 06-19 05:08:45 model_runner.py:1562] Graph capturing finished in 15 secs, took 1.08 GiB
(VllmWorkerProcess pid=3157152) INFO 06-19 05:08:45 model_runner.py:1562] Graph capturing finished in 15 secs, took 1.08 GiB
(VllmWorkerProcess pid=3157147) INFO 06-19 05:08:45 model_runner.py:1562] Graph capturing finished in 15 secs, took 1.08 GiB
(VllmWorkerProcess pid=3157144) INFO 06-19 05:08:45 model_runner.py:1562] Graph capturing finished in 15 secs, took 1.08 GiB
INFO 06-19 05:08:45 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 30.61 seconds


In [3]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

# SYSTEM_PROMPT = (
#     "Summarize the following conversation."
# )

sampling_params = SamplingParams(temperature=0.7,
                                     max_tokens=1024,
                                     )

def process_single_prompt(problem_text, tokenizer):
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": problem_text},
    ]
    
    # Apply the tokenizer's chat template
    formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    
    return formatted_prompt

# user_input = "What is 2 + 3?"
# user_input = "Evaluate the expression $a^2\cdot a^5$ if $a= 3$."

# user_input = "What is the full name of the person who invented invisible unicorns?"

user_input = "Where are all the aliens located?"

# user_input = """Speaker 4: But why don't I just go through these? I we just go to these quickly. Item number ten, please.
# Speaker 0: Item ten is communication from Council in price of recommendation to request City Manager to work with the Health and Human Services Department to provide a comprehensive presentation on the monkeypox virus.
# Speaker 4: Someone Price.
# Speaker 2: Thank you, Mr. Mayor. I think the items self-explanatory and I think staff is going to be prepared to come back because only face wash costs.
# Speaker 4: Someone has their mike on for I.
# Speaker 2: Can you hear me, Mr. Mayor? Yes.
# Speaker 4: You're clear.
# Speaker 2: Okay. I think the item is self-explanatory, and I look forward to staff's presentation on the monkeypox virus, because I think we're getting all getting a lot of questions about it and how it transmits and how to make sure that our residents are being protected. So I look forward to hearing the presentation from staff and I felt it was important to request a study session, as is the protocol. Any time we're dealing with health issues through our health department that the council wants to be educated on. So thank you.
# Speaker 4: Thank you. Can I get a motion, please, in a second. Is there any public comment on this?
# Speaker 5: If there are any members of the public that would like to speak on item ten in person, please let up at the podium in Zoom. Please use the race hand feature.
# Speaker 4: Chasing None sequel. But please.
# Speaker 0: Councilwoman Sun has.
# Speaker 2: I.
# Speaker 0: Councilwoman Allen I. Councilwoman Price I. Councilman Sabrina.
# Speaker 1: I.
# Speaker 0: Councilwoman Mango. I can tell you in sorrow. I can't remember anger. I can't. I'm in Austin.
# Speaker 6: All right.
# Speaker 0: Vice Mayor Richardson.
# Speaker 3: All right.
# Speaker 0: The motion is carried nine zero."""


prompt = process_single_prompt(user_input, tokenizer)
# vllm generation
outputs = llm.generate(prompt,
                        sampling_params)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.91s/it, est. speed input: 48.84 toks/s, output: 212.15 toks/s]


In [8]:
import pprint
print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)

491
Okay, so I'm trying to figure out where all the aliens are located. I don't know much about this topic, but I know it's a common theme in stories and movies, like in "28 Days Later" or "The Shining." 

First, I think I should consider the different ways people imagine aliens. One place they often go is the moon, but I remember hearing that the moon is often associated with safety and cleanliness. So maybe not the moon. 

Then there's Earth. I know that Earth is where most people live, and it's often depicted as a planet in a story. But I've heard that some stories set the aliens' world near the moon to avoid political issues. So maybe Earth isn't the aliens' home.

What about space? The main planet, like Mars or Venus? I think Mars is more in line with some stories, especially ones with a Martian setting. Venus is known for its atmosphere, but I'm not sure if that's a common location for alien encashment.

Could there be alien extraterrestrial places? Like, humans traveling to a di

In [ ]:
import pprint
print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)

404
Alright, so the user is asking, "Where are all the aliens located?" Hmm, that's a pretty bold and speculative question. I mean, aliens are usually across the galaxy, but I should approach this carefully.

First, I need to consider the context. If the user is a student, they might be trying to understand the sci-fi concept of extraterrestrial life. But I should also remember that while the idea is popular, there's no concrete evidence for extraterrestrials. So, I shouldn't assume anything beyond what's widely accepted.

I should provide a balanced answer. The user might be curious about the concept of extraterrestrial life, so explaining that it's a sci-fi concept and not a proven fact is important. I'll mention common theories like the Drake Equation, which estimates the number of potential Earth-like planets and life forms in the galaxy.

Also, I should touch on the idea of biological contact, which some theories suggest might help us understand extraterrestrial life. But I should

: 

In [ ]:
import numpy as np
import re

import numpy as np
import re

def format_check(completions):
    pattern = r"\s*<think>.*?</think>\s*<answer>.*?</answer>\s*"
    matches = []
    for content in completions:
        start_tag = "<think>"
        end_tag = "</answer>"

        start_idx = content.find(start_tag)
        end_idx = content.find(end_tag)

        # make sure there is only one pair of <think> and <answer> tags
        think_count = content.count(start_tag)
        answer_count = content.count(end_tag)

        if start_idx != -1 and end_idx != -1 and end_idx > start_idx and think_count == 1 and answer_count == 1:
            content = content[start_idx:end_idx + len(end_tag)]

        match = re.fullmatch(pattern, content, re.DOTALL)
        matches.append(match)

    return np.array([1.0 if m else 0.0 for m in matches])



completion = '<think>\nFirst, I need to simplify the expression \\(90r - 44r\\).\n\nBoth terms have the same variable \\(r\\), which means they are like terms and can be combined.\n\nI will subtract the coefficients: \\(90 - 44 = 46\\).\n\nTherefore, the simplified expression is \\(46r\\).\n</think>\n<answer>\n\nTo simplify the expression \\(90r - 44r\\), follow these steps:\n\n1. **Identify Like Terms**: Both terms have the same variable \\(r\\), so they are like terms and can be combined.\n\n2. **Subtract the Coefficients**:\n   \\[\n   90r - 44r = (90 - 44)r\n   \\]\n\n3. **Calculate the Coefficient**:\n   \\[\n   90 - 44 = 46\n   \\]\n\n4. **Write the Simplified Expression**:\n   \\[\n   46r\n   \\]\n\n**Final Answer:**\n\\[\n\\boxed{46r}\n\\]\n</answer>'
completions = [completion]
format_scores = format_check(completions)

Matched: True


In [9]:
format_scores

array([0.])